# Face Verification using Siamese Network

## Importing Dependencies

In [8]:
#Standard Dependencies
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random


In [9]:
#importing tensorflow Modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Conv2D,MaxPooling2D,Input,Flatten,Dense
import tensorflow as tf

## Folder Structure & Paths 

In [10]:
POS_PATH=os.path.join('data','positive')
NEG_PATH=os.path.join('data','negative')
ANC_PATH=os.path.join('data','anchor')

In [7]:
#MAke dIrectories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [8]:
!tar -xf lfw.tgz

In [11]:
#Moving the lfw dataset into the negative 
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw',directory)):
        EX_PATH=os.path.join('lfw',directory,file)
        NEW_PATH=os.path.join(NEG_PATH,file)
        os.replace(EX_PATH,NEW_PATH)

In [3]:
import uuid

### Collecting images data Positive & Anchor

In [7]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame=cap.read()
    
    #Getting 250x250 pixel frame
    frame=frame[120:120+250,200:200+250]
    
    
    #Collecting anchor images
    if cv2.waitKey(1) & 0XFF==ord('a'):
        imgname=os.path.join(ANC_PATH,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
    
    
    #Collecting positive images
    if cv2.waitKey(1) & 0XFF==ord('p'):
        imgname=os.path.join(POS_PATH,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
    
    cv2.imshow('Image Collection',frame)
    
    if cv2.waitKey(1) & 0XFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 3 Load & Preprocess Images

### 3.1 Get directories 

In [13]:
anchor=tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300)
positive=tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)
negative=tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

### 3.2 Preprocess- Scale & Resize

In [14]:
def preprocess(file_path):
    byte_img=tf.io.read_file(file_path)
    img=tf.io.decode_jpeg(byte_img)
    img=tf.image.resize(img,(100,100))
    img=img/255.0
    return img

### 3.3 Creating labelled Dataset

In [15]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data=positives.concatenate(negatives)

### 3.4 Build Train & Test Partition

In [49]:
def preprocess_twin(input_image,validation_image,label):
    return (preprocess(input_image), preprocess(validation_image),label)

In [50]:
#Building data pipeline
data=data.map(preprocess_twin)
data=data.cache()
data=data.shuffle(buffer_size=1024)

In [51]:
#Training Partition
train_data=data.take(round(len(data)*.7))
train_data=train_data.batch(16)
train_data=train_data.prefetch(8)

In [52]:
#Testing Partition
test_data=data.skip(round(len(data)*.7))
test_data=test_data.take(round(len(data)*.3))
test_data=test_data.batch(16)
test_data=test_data.prefetch(8)

## 4 Model Engineering 

### 4.1 Build Embedding Layer

In [22]:
# Embedding Layer
def make_embedding():
    inp=Input(shape=(100,100,3),name='input_image')
    
    #First Block
    c1=Conv2D(64,(10,10),activation='relu')(inp)
    m1=MaxPooling2D(64,(2,2),padding='same')(c1)
    
    #Second block
    c2=Conv2D(128,(7,7),activation='relu')(m1)
    m2=MaxPooling2D(64,(2,2),padding='same')(c2)
    
    #third block
    c3=Conv2D(128,(4,4),activation='relu')(m2)
    m3=MaxPooling2D(64,(2,2),padding='same')(c3)
    
    #Final Embedding block
    c4=Conv2D(256,(4,4),activation='relu')(m3)
    f1=Flatten()(c4)
    d1=Dense(4096,activation='sigmoid')(f1)
    
    
    
    return Model(inputs=[inp],outputs=[d1],name='embedding')
    

In [23]:
embedding=make_embedding()
#Use embedding.summary() to checkout whole model
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0 

### 4.2 L1 Distance Layer

In [24]:
#Creating a class to calculate L1 distance using tensorflow
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
        
    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding-validation_embedding)

### 4.3 Creating Siamese Model 

In [41]:
def make_siamese_model():
    
    #Anchor inputs
    input_image=Input(name='input_image',shape=(100,100,3))
    
    #Validation image Inputs
    validation_image=Input(name='validation_image',shape=(100,100,3))
    
    siamese_layer=L1Dist()
    siamese_layer._name='distance'
    distance=siamese_layer(embedding(input_image),embedding(validation_image))
    
    #classification layer
    classifier=Dense(1,activation='sigmoid')(distance)
    
    return Model(inputs=[input_image,validation_image],outputs=classifier,name='SiameseNetwork')

In [42]:
siamese_model=make_siamese_model()

## 5 Training

In [43]:
binary_cross_loss=tf.losses.BinaryCrossentropy()
opt=tf.keras.optimizers.Adam(1e-4)

### 5.2 Establish checkpoints

In [44]:
checkpoint_dir='/training_checkpoints'
checkpoint_prefix=os.path.join(checkpoint_dir,'ckpt')
checkpoint=tf.train.Checkpoint(opt=opt,siamese_model=siamese_model)

### 5.3 Train Step Function

In [54]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        
        X=batch[:2] # Get anchor Postive/Negtive images

        y=batch[2] # Get the labels

        yhat=siamese_model(X, training=True)

        loss=binary_cross_loss(y, yhat)

        #Calculate Gradient
        grad=tape.gradient(loss,siamese_model.trainable_variables)
        
        opt.apply_gradients(zip(grad,siamese_model.trainable_variables))
        
        
        #Returning the losses
        return loss
    
    
    
    

### 5.4 Build Train 

In [55]:
def train(data,EPOCHS):
    for epoch in range(1,EPOCHS+1):
        print('\n Epochs {}/{}'.format(epoch,EPOCHS))
        progbar=tf.keras.utils.Progbar(len(data))
        
        #Looping through each batch
        for idx,batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
            
        #Saving the checkpoints
        if epoch%10==0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [56]:
EPOCHS=50


In [57]:
train(train_data,EPOCHS)


 Epochs 1/50
27/27 [==============================] - 586s 22s/step

 Epochs 2/50
27/27 [==============================] - 593s 22s/step

 Epochs 3/50
27/27 [==============================] - 740s 28s/step

 Epochs 4/50
27/27 [==============================] - 602s 22s/step

 Epochs 5/50
27/27 [==============================] - 887s 33s/step

 Epochs 6/50
27/27 [==============================] - 577s 21s/step

 Epochs 7/50
27/27 [==============================] - 577s 21s/step

 Epochs 8/50
27/27 [==============================] - 582s 22s/step

 Epochs 9/50
27/27 [==============================] - 604s 22s/step

 Epochs 10/50
27/27 [==============================] - 659s 24s/step


PermissionDeniedError: Failed to create a directory: /; Permission denied [Op:MergeV2Checkpoints]